# Run the final state algorithm on a ROOT HSDATA file

In [1]:
gROOT->ProcessLine(".x LoadK2.C+");

%%%%%%%%%%%%%%%%%%%%%%%%%    THSParticle
%%%%%%%%%%%%%%%%%%%%%%%%%    EventInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    RunInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    THSWeights
%%%%%%%%%%%%%%%%%%%%%%%%%    DataManager
%%%%%%%%%%%%%%%%%%%%%%%%%    LundReader
&&&&&&&&&&&&&&&&&&&&&&&&&&&& HSKinematics
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Cuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Combitorial
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleIter
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Topology
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FinalState
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FiledTree
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreeData
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TopoActionManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& VarsParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalID
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCutsManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalIDManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS/CLASTrigger
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS12/CLAS12

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.

In [2]:
 K2 fs("Beam","Beam");
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

Charge ID particle are  -22 
Incusive particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
Charge ID particle are  -22 
Incusive particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
Charge ID particle are  -22 
Incusive particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
Charge ID particle are  -22 
Incusive particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 Topology::Print() : 0
    particles = -10000 -22 10000 10000 
You can have any number of the following particles : 
-22 
The following particle are identified by pdg code : 
-22 
 ParticleIte

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [3]:
auto output = new HS::FiledTree("tt","testcuts.root");
fs.FinalStateOutTree(output->Tree()); //connect ouput tree to final state branches

Here we add the cuts for each particle. This is done via a HS::Cuts object which gets propogated to each particle in each topology. To specify any type of cut behaviour you need to generate your own Cuts Method. Here we just use a predefined delta time cut but you could make your won class which has momentum dependence as well.

In [4]:
ParticleCutsManager cutsman;
auto beamCut=make_shared<DeltaTimeCut>(2); //7ns delta time cut
auto otherCut=make_shared<DeltaTimeCut>(1); //10ns cut for all other particles
cutsman.AddParticleCut("Beam",beamCut); //assign to manager
cutsman.SetDefaultCut(otherCut); //assign to manager
cutsman.ConfigureCuts(&fs);    //propogate the cuts to topologies in fs

Info ParticleCutsManager AddParticleCut set cut for 0
Info ParticleCutsManager AddParticleCut set cut for -22


Now register the cuts manager as a post topology function action. This will be applied after the Topo_0 etc functions when start times etc have been determined. 

In [5]:
fs.RegisterPostTopoAction(&cutsman);

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.

In [6]:
 //create datamanager
  auto dm=std::make_shared<DataManager>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/w/work1/jlab/hallb/HS_g8/1_9_list/flist_048088.dat.root");
  dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

Info in <DataManager::InitChain>:  Will proceess all 1 files in chain


DataManager::InitReader set branch 0x7fd0c408cd48 0x7fd0c408cd48


Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [ ]:
cout<<"Number of Events to process "<<chain.GetEntries()<<endl;

fs.ProcessData(); //No number give, analyse all events in chain
cout<<"Done "<<dm->GetEntry()<< " events "<<endl;

Number of Events to process 1111760



 *** Break *** segmentation violation



There was a crash.
This is the entire stack trace of all threads:

Thread 9 (Thread 0x7fd0eab39700 (LWP 14072)):
#0  0x00007fd10a904183 in epoll_wait () from /lib64/libc.so.6
#1  0x00007fd0ed2f9c21 in ?? () from /usr/lib64/python2.7/site-packages/zmq/backend/cython/../../.libs/libzmq-f7ef747e.so.5.1.5
#2  0x00007fd0ed316391 in ?? () from /usr/lib64/python2.7/site-packages/zmq/backend/cython/../../.libs/libzmq-f7ef747e.so.5.1.5
#3  0x00007fd0ed33df84 in ?? () from /usr/lib64/python2.7/site-packages/zmq/backend/cython/../../.libs/libzmq-f7ef747e.so.5.1.5
#4  0x00007fd10b2e2e25 in start_thread () from /lib64/libpthread.so.0
#5  0x00007fd10a903bad in clone () from /lib64/libc.so.6

Thread 8 (Thread 0x7fd0ea338700 (LWP 14073)):
#0  0x00007fd10a904183 in epoll_wait () from /lib64/libc.so.6
#1  0x00007fd0ed2f9c21 in ?? () from /usr/lib64/python2.7/site-packages/zmq/backend/cython/../../.libs/libzmq-f7ef747e.so.5.1.5
#2  0x00007fd0ed316391 in ?? () fro

And remember to save the tree!

In [ ]:
//delete output;

In [8]:
%jsroot on

In [9]:
TCanvas c2;
output->Tree()->Draw("MissMass>>(100,-1,2)","Topo==1");
output->Tree()->Draw("MissMass","Topo==0","same");
output->Tree()->Draw("MissMass","Topo==2","same");
output->Tree()->Draw("MissMass","Topo==3","same");


In [10]:
c2.Draw();